# Assignment 2: Feature Selection for Attrition / Burnout Prediction

# Group 14: Rylie Ramos-Marquez, Derek Atabayev, Vishnu Garigipati

From the previous assignment, we know that the best method is gradient boosting of trees, specifically with 200 boosting rounds.

Why it's the best:

* High F1-score = 0.9319 which far surpasses other models (better by at least 0.1 = 10%), and is also better than simple KNN/Decision trees model

* Shallow tree structure (max depth = 5) which prevents overfitting

* Consistent performance with a tight 95% confidence interval of [0.855171, 0.893952] which is a very small range, meaning the performance is consistent and strong

The final model is saved as a Pickle format for easy retrieval

In [2]:
# imports for the project

import joblib # since joblib.dump was used to save the model
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.calibration import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score

import numpy as np

In [3]:
# retrieve final_model_14.pkl from ../final_model_14.pkl

model = joblib.load('../final_model_14.pkl')

# check if the model has been loaded correctly
print(model)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  Index(['hrs', 'absences', 'JobInvolvement', 'PerformanceRating',
       'EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance', 'Age',
       'DistanceFromHome', 'Education', 'EmployeeID', 'JobLevel',
       'MonthlyIncome', 'NumCompaniesWorked', 'PercentSalaryHike',
       'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear',
       'YearsAtCompany', 'YearsSinceLastPromotion', 'YearsWithCurrManager'],
      dtype='object'))])),
                ('classifier',
                 GradientBoostingClassifier(max_d

Looks like the model has been loaded successfully. We can see that the feature names are correct and familiar, so the preprocessing and classifier are intact. All the gradient boosting parameters are there too.

For the grid search later, we will need our training data. We can pull that now too.

We added a cell to the notebook from assignment 1 to export this data to pickle files

Our ideal split from assignment one was 80/20 training/testing

In [4]:
# save data X and y from pickle

X = joblib.load('./X.pkl')
y = joblib.load('./y.pkl')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100545358)

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)


We can check which pairs of indices have a high correlation.

A correlation score close to one would suggest that through feature selection we can remove one element of the pair (since they are contributing similar predictive power)

In [5]:
correlation_matrix = X.corr(numeric_only=True)
high_corr_pairs = correlation_matrix.where((correlation_matrix > 0.7) & (correlation_matrix < 1))

for col in high_corr_pairs.columns:
    high_corr_indices = high_corr_pairs.index[high_corr_pairs[col].notnull()]
    for idx in high_corr_indices:
        print(f"High correlation: {col} and {idx} -> {correlation_matrix.loc[idx, col]:.2f}")

High correlation: PerformanceRating and PercentSalaryHike -> 0.79
High correlation: PercentSalaryHike and PerformanceRating -> 0.79
High correlation: YearsAtCompany and YearsWithCurrManager -> 0.76
High correlation: YearsWithCurrManager and YearsAtCompany -> 0.76


# Adding feature selection

* SelectKBest and criterion f_classif

In [6]:
# add feature selection with SelectKBest and f_classif to the pipeline

# score_func is the function used to evaluate the importance of each feature
# f_classif is a default value for the score_func parameter, calculates ANOVA F-Value

# Extract components from the existing pipeline
preprocessor = model.named_steps['preprocessor']
classifier = model.named_steps['classifier']

# create pipeline 1: feature selection with SelectKBest and criterion f_classif

pipeline_f_classif = Pipeline([
    ('preprocessor', preprocessor),
    ('selector', SelectKBest(f_classif)),
    ('classifier', classifier)
])

pipeline_mutual_info_classif = Pipeline([
    ('preprocessor', preprocessor),
    ('selector', SelectKBest(mutual_info_classif)),
    ('classifier', classifier)
])


We place the selector function in the pieline after preprocessing (encoding the values) but before it is classified (through the gradient boosting model)

Performing grid search to tune the number of features to be selected (k parameter)

  * We need to use an array to test different values of k, since we have 21 features, we can check check all of them through an array builder (loop through 1 to 21)
  * This will tell us whether the higher side or lower side is best

In [8]:
# performing grid search to find the best parameters for the model, best metric is accuracy

# define the parameters for the grid search

# parameters for choosing a value of k
param_grid = {
    'selector__k': [i for i in range(1,22)]
}

def perform_grid_search(param_grid):
    # create grid search object for pipeline 1
    grid_search_f_classif = GridSearchCV(pipeline_f_classif, param_grid, cv=5, n_jobs=-1, verbose=0)
    # in the previous assignment, we used 5-fold cross validation because the dataset is small, and we want to make sure that the model is not overfitting
    # that approach worked well and so we can use it for HPO of the 'k' parameter too

    # creating grid search object for pipeline 2
    grid_search_mutual_info_classif = GridSearchCV(pipeline_mutual_info_classif, param_grid, cv=5, n_jobs=-1, verbose=0)

    # fit the grid search objects to the data
    grid_search_f_classif.fit(X_train, y_train_encoded)
    grid_search_mutual_info_classif.fit(X_train, y_train_encoded)

    # print the optimal k in both cases
    best_k_f_classif = grid_search_f_classif.best_params_['selector__k']
    print(f'Optimal value of k for f_classif: {best_k_f_classif}')

    best_k_mutual_info_classif = grid_search_mutual_info_classif.best_params_['selector__k']
    print(f'Optimal value of k for mutual_info_classif: {best_k_mutual_info_classif}')

    print("\nAverage accuracy for each k (f_classif):")
    for mean, params in zip(grid_search_f_classif.cv_results_['mean_test_score'], grid_search_f_classif.cv_results_['params']):
        print(f"k = {params['selector__k']}: Average Accuracy = {mean:.4f}")

    # Print average accuracy for each k for mutual_info_classif
    print("\nAverage accuracy for each k (mutual_info_classif):")
    for mean, params in zip(grid_search_mutual_info_classif.cv_results_['mean_test_score'], grid_search_mutual_info_classif.cv_results_['params']):
        print(f"k = {params['selector__k']}: Average Accuracy = {mean:.4f}")
    return grid_search_f_classif, grid_search_mutual_info_classif

# Call the function with the parameter grid
grid_search_f_classif, grid_search_mutual_info_classif = perform_grid_search(param_grid)


Optimal value of k for f_classif: 19
Optimal value of k for mutual_info_classif: 17

Average accuracy for each k (f_classif):
k = 1: Average Accuracy = 0.6202
k = 2: Average Accuracy = 0.6526
k = 3: Average Accuracy = 0.6921
k = 4: Average Accuracy = 0.7719
k = 5: Average Accuracy = 0.7886
k = 6: Average Accuracy = 0.8132
k = 7: Average Accuracy = 0.8246
k = 8: Average Accuracy = 0.8333
k = 9: Average Accuracy = 0.8368
k = 10: Average Accuracy = 0.8342
k = 11: Average Accuracy = 0.8325
k = 12: Average Accuracy = 0.8456
k = 13: Average Accuracy = 0.8535
k = 14: Average Accuracy = 0.8421
k = 15: Average Accuracy = 0.8579
k = 16: Average Accuracy = 0.8526
k = 17: Average Accuracy = 0.8544
k = 18: Average Accuracy = 0.8640
k = 19: Average Accuracy = 0.8763
k = 20: Average Accuracy = 0.8763
k = 21: Average Accuracy = 0.8746

Average accuracy for each k (mutual_info_classif):
k = 1: Average Accuracy = 0.7693
k = 2: Average Accuracy = 0.8316
k = 3: Average Accuracy = 0.8412
k = 4: Average Acc

The optimal 'k' value for each feature selection is going to different across many trials.

Reasons: 

* The data may be shuffled differently, which leads to variation in splits and a change in performance for different k.

* Since the dataset is small, the splitting can affect the k

* Since it is a small dataset, the accuracy difference is small, but can still result in a different recommendation 

 * see the output for the accuracy 

We can average across many trials to find the true "optimal" k value

Perform 30 runs and take the average (long execution time, but optimized results)

In [15]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

def perform_multiple_grid_search(param_grid, X_train, y_train_encoded, n_runs=20):
    best_ks_f_classif = []
    best_ks_mutual_info_classif = []
    
    # Store accuracies for each k
    accuracy_f_classif = {k: [] for k in param_grid['selector__k']}
    accuracy_mutual_info_classif = {k: [] for k in param_grid['selector__k']}
    
    # Perform grid search over multiple runs
    for _ in range(n_runs):
        # Create cross-validation strategy
        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=None)
        
        # Define grid search objects
        grid_search_f_classif = GridSearchCV(pipeline_f_classif, param_grid, cv=cv, n_jobs=-1, verbose=0)
        grid_search_mutual_info_classif = GridSearchCV(pipeline_mutual_info_classif, param_grid, cv=cv, n_jobs=-1, verbose=0)

        # Fit the grid search objects to the data
        grid_search_f_classif.fit(X_train, y_train_encoded)
        grid_search_mutual_info_classif.fit(X_train, y_train_encoded)

        # Get optimal k for each method
        best_k_f_classif = grid_search_f_classif.best_params_['selector__k']
        best_k_mutual_info_classif = grid_search_mutual_info_classif.best_params_['selector__k']
        
        best_ks_f_classif.append(best_k_f_classif)
        best_ks_mutual_info_classif.append(best_k_mutual_info_classif)

        # Record accuracies for each k
        for mean, params in zip(grid_search_f_classif.cv_results_['mean_test_score'], grid_search_f_classif.cv_results_['params']):
            accuracy_f_classif[params['selector__k']].append(mean)

        for mean, params in zip(grid_search_mutual_info_classif.cv_results_['mean_test_score'], grid_search_mutual_info_classif.cv_results_['params']):
            accuracy_mutual_info_classif[params['selector__k']].append(mean)
    
    # Calculate average optimal k and accuracy
    avg_best_k_f_classif = np.mean(best_ks_f_classif)
    avg_best_k_mutual_info_classif = np.mean(best_ks_mutual_info_classif)

    print(f'Average optimal value of k for f_classif over {n_runs} runs: {avg_best_k_f_classif}')
    print(f'Average optimal value of k for mutual_info_classif over {n_runs} runs: {avg_best_k_mutual_info_classif}')
    
    print("\nAverage accuracy for each k (f_classif):")
    for k, accuracies in accuracy_f_classif.items():
        print(f"k = {k}: Average Accuracy = {np.mean(accuracies):.4f} ± {np.std(accuracies):.4f}")
    
    print("\nAverage accuracy for each k (mutual_info_classif):")
    for k, accuracies in accuracy_mutual_info_classif.items():
        print(f"k = {k}: Average Accuracy = {np.mean(accuracies):.4f} ± {np.std(accuracies):.4f}")
    print(f'\n\n best k_f {best_ks_f_classif}')
    print(f'\n\n best mutual info {best_ks_mutual_info_classif}')
    return avg_best_k_f_classif, avg_best_k_mutual_info_classif, accuracy_f_classif, accuracy_mutual_info_classif

# Example usage:
param_grid = {'selector__k': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]}
avg_best_k_f_classif, avg_best_k_mutual_info_classif, accuracy_f_classif, accuracy_mutual_info_classif = perform_multiple_grid_search(param_grid, X_train, y_train_encoded, n_runs=30)


KeyboardInterrupt: 

Across the 30 runs, each candidate k value is selected an arbitrary number of times.

Here, we use the array outputted with various k values and find the number of times each value it appears.

Taking the mode of each list gives us the truly optimal k value.

list1 is for f_classif, list2 is for mutual_info_classif criteria - the arrays with the 30 values returned in our trial are listed, but even they may be slightly different across trials.

In [10]:
from collections import Counter

# Get the union of keys from both counters

best_ks_f_classif = [18, 11, 21, 17, 17, 20, 19, 19, 21, 14, 19, 20, 16, 19, 19, 19, 21, 20, 21, 15, 15, 20, 18, 21, 19, 17, 15, 17, 16, 19]
best_ks_mutual_info_classif = [19, 12, 18, 15, 19, 18, 18, 20, 19, 20, 19, 16, 18, 20, 19, 16, 13, 15, 11, 19, 20, 21, 10, 18, 20, 19, 19, 19, 12, 17]
counter1 = Counter(best_ks_f_classif)
counter2 = Counter(best_ks_mutual_info_classif)

# Format counts and sort by values (counts) in descending order
formatted_counts1 = ", ".join(f"{key}: {value}" for key, value in sorted(counter1.items(), key=lambda x: x[1], reverse=True))
formatted_counts2 = ", ".join(f"{key}: {value}" for key, value in sorted(counter2.items(), key=lambda x: x[1], reverse=True))

# Print results
print(f"Counts in list1: {formatted_counts1}")
print(f"Counts in list2: {formatted_counts2}")

Counts in list1: 19: 8, 21: 5, 17: 4, 20: 4, 15: 3, 18: 2, 16: 2, 11: 1, 14: 1
Counts in list2: 19: 9, 18: 5, 20: 5, 12: 2, 15: 2, 16: 2, 13: 1, 11: 1, 21: 1, 10: 1, 17: 1


Optimal k-value (both) is 19.

Evaluate using this on the testing dataset

In [13]:
# Fit with k=19 for both methods
pipeline_f_classif.set_params(selector__k=19)

pipeline_f_classif.fit(X_train, y_train_encoded)

pipeline_mutual_info_classif.set_params(selector__k=19)

pipeline_mutual_info_classif.fit(X_train, y_train_encoded)

# Predict the target values

y_pred_f_classif = pipeline_f_classif.predict(X_test)

y_pred_mutual_info_classif = pipeline_mutual_info_classif.predict(X_test)

# Calculate accuracy and f1-score

accuracy_f_classif = accuracy_score(y_test_encoded, y_pred_f_classif)
f1_f_classif = f1_score(y_test_encoded, y_pred_f_classif, average='weighted')

accuracy_mutual_info_classif = accuracy_score(y_test_encoded, y_pred_mutual_info_classif)
f1_mutual_info_classif = f1_score(y_test_encoded, y_pred_mutual_info_classif, average='weighted')

print(f"Accuracy for f_classif: {accuracy_f_classif:.4f}")
print(f"F1-score for f_classif: {f1_f_classif:.4f}")

print(f"Accuracy for mutual_info_classif: {accuracy_mutual_info_classif:.4f}")
print(f"F1-score for mutual_info_classif: {f1_mutual_info_classif:.4f}")



Accuracy for f_classif: 0.6608
F1-score for f_classif: 0.6598
Accuracy for mutual_info_classif: 0.9476
F1-score for mutual_info_classif: 0.9475


In [14]:
# Evaluate the models obtained with the two pipelines on the testing dataset

# get the best models from the grid search object
best_model_f_classif = grid_search_f_classif.best_estimator_

best_model_mutual_info_classif = grid_search_mutual_info_classif.best_estimator_

# print the accuracy and f1-score of the best models on the testing dataset

y_pred_f_classif = best_model_f_classif.predict(X_test)
y_pred_mutual_info_classif = best_model_mutual_info_classif.predict(X_test)

accuracy_f_classif = accuracy_score(y_test_encoded, y_pred_f_classif)
f1_f_classif = f1_score(y_test_encoded, y_pred_f_classif)

accuracy_mutual_info_classif = accuracy_score(y_test_encoded, y_pred_mutual_info_classif)
f1_mutual_info_classif = f1_score(y_test_encoded, y_pred_mutual_info_classif)

print(f'Accuracy of the best model with f_classif: {accuracy_f_classif}')
print(f'F1-score of the best model with f_classif: {f1_f_classif}')

print(f'Accuracy of the best model with mutual_info_classif: {accuracy_mutual_info_classif}')
print(f'F1-score of the best model with mutual_info_classif: {f1_mutual_info_classif}')




Accuracy of the best model with f_classif: 0.9265734265734266
F1-score of the best model with f_classif: 0.9288135593220339
Accuracy of the best model with mutual_info_classif: 0.9090909090909091
F1-score of the best model with mutual_info_classif: 0.9115646258503401


The best feature selection method is with the criterion: mutual_info_classif

The accuracy and f1-scores in this case are slightly better, but both models are pretty much the same.

In [18]:
# Check which features are actually selected
# We have 21, and k=20, so one has been dropped

all_features = ['hrs', 'absences', 'JobInvolvement', 'PerformanceRating',
       'EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance', 'Age',
       'DistanceFromHome', 'Education', 'EmployeeID', 'JobLevel',
       'MonthlyIncome', 'NumCompaniesWorked', 'PercentSalaryHike',
       'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear',
       'YearsAtCompany', 'YearsSinceLastPromotion', 'YearsWithCurrManager']
# get the selected features from the best model with mutual_info_classif
selected_features_mutual_info_classif = best_model_mutual_info_classif.named_steps['selector'].get_support()

# find the feature that has been dropped
dropped_feature = [feature for feature, selected in zip(all_features, selected_features_mutual_info_classif) if not selected] # zip is used to iterate over two lists at the same time
print(f'The dropped feature is: {dropped_feature}')


The dropped feature is: ['absences']


All features, besides 'absences' are selected. 20 total features

The results are improved as compared to the previous assignment, F1 Score has increased from 0.9320 to 0.9424

Reason is likely that 'absences' might have been so poorly correlated to attrition that it contributed noise, which means the model can improve its predictions after removing it


In [16]:
# Redefine the pipeline with k = 20

pipeline_mutual_info_classif = Pipeline([
    ('preprocessor', preprocessor),
    ('selector', SelectKBest(mutual_info_classif, k=20)), # add this feature selection
    ('classifier', classifier)
])

# Do hyper-parameter tuning with the new pipeline

# define the parameters for the grid search

param_grid = {
    'classifier__n_estimators': [100, 200, 300, 400, 500],
    'classifier__max_depth': [5, 10, 15, 20, 25]
}

# perform grid search
grid_search_mutual_info_classif = GridSearchCV(pipeline_mutual_info_classif, param_grid, cv=5, n_jobs=-1, verbose=0)

grid_search_mutual_info_classif.fit(X_train, y_train_encoded)

# print the optimal parameters

best_params_mutual_info_classif = grid_search_mutual_info_classif.best_params_

print(f'Optimal parameters for the best model with mutual_info_classif: {best_params_mutual_info_classif}')



Optimal parameters for the best model with mutual_info_classif: {'classifier__max_depth': 10, 'classifier__n_estimators': 500}


Evaluate whether results are further improved

As usual, we are using accuracy as the metric to evaluate the model, because it is a classification problem and the classes are balanced

But also, we add the F1-score to have a more complete view of the model's performance.

In [17]:
# Check if results are further improved

# get the best model from the grid search object
best_model_mutual_info_classif = grid_search_mutual_info_classif.best_estimator_

# predict the target values
y_pred_mutual_info_classif = best_model_mutual_info_classif.predict(X_test)

# calculate accuracy and f1-score
accuracy_mutual_info_classif = accuracy_score(y_test_encoded, y_pred_mutual_info_classif)
f1_mutual_info_classif = f1_score(y_test_encoded, y_pred_mutual_info_classif)

print(f'Accuracy of the best model with mutual_info_classif after HPO: {accuracy_mutual_info_classif}')
print(f'F1-score of the best model with mutual_info_classif after HPO: {f1_mutual_info_classif}')



Accuracy of the best model with mutual_info_classif after HPO: 0.9545454545454546
F1-score of the best model with mutual_info_classif after HPO: 0.9559322033898305


Results are further improved, mostly because we eliminated the redundant feature ('absences').

  * This reduces the number of dimensions which reduces the risk of overfitting, so the model can perform better on previously unseen data

  * With a more reliable (smaller) set of features, the use of 5-fold cross-validation ensures that there are multiple subsets of data that are being trained and tested on a more relevant model

  * The number of trees is very high, as HPO chooses but this doesn't cause overfitting because the growth of each of those many trees is controlled (max_depth = 10)


We went back and saved the competition_data (already preprocessed) in a pickle file from Assignment 1

In [20]:
# Make predictions on competition data

# load the competition data

X_competition = joblib.load('./competition_data.pkl')

# predict the target values for the competition data using the best model with mutual_info_classif after HPO

y_pred_competition = best_model_mutual_info_classif.predict(X_competition)

# save the predictions to a file

joblib.dump(y_pred_competition, './attrition_predictions_14.csv')

# save the model

joblib.dump(best_model_mutual_info_classif, './final_model_14.pkl')



['./final_model_14.pkl']